In [1]:
import torch
from torchvision.transforms.functional import to_pil_image
from src.model import RefineNet
from src.score_matching import linear_noise_scale


device = torch.device('cuda:1')
noise_steps = 10

model = RefineNet(
    in_channels=1,
    hidden_channels=(64, 128, 256, 512),
    n_noise_scale=noise_steps
).to(device)

model.load_state_dict(torch.load('ckpts/refinenet_mnist.pth'))
noise_scales = linear_noise_scale(start=1., end=0.01, length=10).to(device)

In [11]:
from src.langevin_dynamics import sample


samples = sample(model, (8, 1, 28, 28), noise_scales, device, n_steps=200, eps=1e-5)

100%|██████████| 10/10 [01:38<00:00,  9.85s/it]


In [17]:
samples = torch.clamp(samples, 0., 1.)
to_pil_image(samples[2])